# Api Get Commodities

In [ ]:
# Imports

import pandas as pd
import requests
import re

from datetime import datetime

In [ ]:
# Read access token

with open('../api_access_token.txt', 'r') as file:
   access_token = file.read()

In [ ]:
# Get commodity auctions data

def get_commodities(access_token):
   search = f'https://eu.api.blizzard.com/data/wow/auctions/commodities?namespace=dynamic-eu&locale=en_GB&access_token={access_token}'
   response = requests.get(search)
   return response.json()['auctions']

commodities_data = get_commodities(access_token)

commodities_df = pd.DataFrame(commodities_data)

commodities_df.head()

In [ ]:
# Add timestamp

now = datetime.now()

date_str = now.strftime('%Y-%m-%d %H:%M:%S')

commodities_df['timestamp'] = date_str

commodities_df['timestamp'] = pd.to_datetime(commodities_df['timestamp'])

commodities_df.head()

In [ ]:
# Extract item ids

commodities_df['item'] = commodities_df['item'].apply(lambda x: x['id'])

commodities_df.head()

In [ ]:
# Rename columns

commodities_df.rename(columns = {'id': 'auction_id', 'item': 'item_id'}, inplace = True)

commodities_df.head()

In [ ]:
# Save commodities data to file

def stamp_string(timestamp):
   day = re.findall('-(\d.) ', str(timestamp))[0]
   hour = re.findall(' (\d.):', str(timestamp))[0]
   return 'd' + day + '_h' + hour
   
file_stamp = stamp_string(commodities_df.iloc[:1, 5])

commodities_df.to_csv(f'../data/commodities/{file_stamp}_commodities.csv')